In [ ]:
# Install relevant libraries
!pip install numpy matplotlib pandas

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Problem Statement
In this section we are going to the implement the polupar contexual algorithm "LinUCB". 
Paper link: [LinUCB](http://rob.schapire.net/papers/www10.pdf)

In this paper, the authors focus on choosing which articles to put on the homepage of a site. Say, for example, they had 3 articles but only space for 1, they could use the LinUCB algorithm to find which of the articles is best. More interestingly, if they had some features about their users–say if they had clicked on a sports article in the past or if they had clicked on a politics article in the past–the algorithm can take that into account and find which articles are best for people given their past click behaviors.

We will implement the simpler version of LinUCB which makes two assumptions: first, our observations are i.i.d. and second, the arms are independent(Parameters are not shared across the arms).

### Reading the Data
1st column represents the arm which were be shown to user. Values between (1,10)
2nd column represents the reward either 0 or 1 for choosing that perticular arm.
From 3rd onwards Features of arm and user combined.

In [ ]:
filepath = "/home/ashutosh/Desktop/linucb_data.csv"  # Replace this filepath with yours.
data = np.genfromtxt(filepath,delimiter=' ')

In [ ]:
data.shape
data[:4,:]  # Visualize the data.

In [ ]:
# Main parameters for algorithm
n_points = data.shape[0] # total data points
n_arms = 10   # number of arms
n_features = 100 # number of features


In [ ]:
context=data[:,2:]            # Context  Matrix (data), features starts from 3rd column.

#context.shape
theta=np.random.random( (n_arms,n_features) ) - 0.5      # real theta, what we will try to guess
#theta.shape
P=context.dot(theta.T)
optimal=np.array(P.argmax(axis=1), dtype=int)  # Maximum value among all arms to select best one.

#plt.title("Distribution of ideal arm choices")
#plt.hist(optimal,bins=range(0,n_arms));

### Implementing Main Algorithm: LinUCB Algorithm with Disjoint Linear Models.

In [ ]:
using actual theta to figure out rewardeps = 0.2
choices = np.zeros(n_points)
rewards = np.zeros(n_points)
explore = np.zeros(n_points)
norms   = np.zeros(n_points)

b = np.zeros((n_arms,n_features))
#b = np.zeros_like(theta)
A  = np.zeros((n_arms,n_features,n_features))

#print(temp.shape)
#print( ba.shape) 
#print(th.shape)

# Making Identity matrix for all new arms at initial times.
for i in range(n_arms):
    A[i] = np.identity(n_features)
    
theta_hat = np.zeros_like(theta) # Current best prediction
p = np.zeros(n_arms)   # Mean + Confidence Interval for arms.
alpha = 0.2 # constant to hold tight inequality.

for i in range(n_points):
    current_context = context[i]  # current context vector for t=i
    
    for a in range(n_arms):
        inv_A = np.linalg.inv(A[a])   # inverse of matrix A
        theta_hat[a] = inv_A.dot(b[a])  # Calculating theta_hat
        variance = current_context.dot(inv_A).dot(current_context)
        a_upper_ci = alpha*np.sqrt(variance)  # Upper confidence interval
        a_mean = theta_hat[a].dot(current_context)
        p[a] = a_mean + a_upper_ci
        
    norms[i]       = np.linalg.norm(theta_hat - theta,'fro')    # diagnostic, are we converging ?
        
    # Let's not be biased with tiebreaks, but add in some random noise
    p= p + ( np.random.random(len(p)) * 0.000001)
    choices[i] = p.argmax()   # choose the highest, line 11
    rewards[i] = theta[int(choices[i])].dot(current_context)    # Calculating reward for current context with selected arm choice[i]
    
    # Update A and b (Incremental Fashion)
    A[int(choices[i])] += np.outer(current_context,current_context)
    b[int(choices[i])] += rewards[i]*current_context

### Ploting the Cumulative Regret and the norm of |true reward - estimated reward|

In [ ]:
plt.figure(1,figsize=(10,5))
plt.subplot(121)
plt.plot(norms);
plt.title("Frobenius norm of estimated theta vs actual");

regret=(P.max(axis=1) - rewards)
plt.subplot(122)
plt.plot(regret.cumsum())
plt.title("Cumulative Regret");